# Model Research

Debido a la naturaleza del dataset, se probarán los siguientes modelos:
- RandomForestCassifier: Debido a su adaptabilidad en problemas de clasificación multiclase.
- XGBoostClassifieer: 
- Adaboost